In [ ]:
from luminoso_api import LuminosoClient
from pack64 import unpack64
from tableau_export import pull_lumi_data

import numpy as np

In [ ]:
client, docs, topics, terms, subsets, drivers, skt, themes = pull_lumi_data('d53m338v', 'pr3n28bw')

In [ ]:
print(len(unpack64(topics[0]['vector'])))

In [ ]:
print(len(unpack64(docs[0]['vector'])))

In [ ]:
np.dot(unpack64(topics[0]['vector']), unpack64(docs[0]['vector']))

In [ ]:
len(terms)

In [ ]:
topics[0]

In [ ]:
%run tableau_export.py d53m338v pr3n28bw -t 150

In [ ]:
docs = client.get('docs')

In [ ]:
docs[0]

In [1]:
%run tableau_export.py demo qdwr5

Extracting Lumi data...
Creating doc table...


/usr/local/lib/python3.6/site-packages/se_code-0.1-py3.6.egg/se_code/fuzzy_logic.py:38: RuntimeWarning: invalid value encountered in double_scalars


Writing to file doc_table.csv.
Writing to file xref_table.csv.
Creating themes table...
Writing to file themes_table.csv.
Creating subset key terms table...
Writing to file skt_table.csv.
Writing to file drivers_table.csv.
Writing to file trends_table.csv.
Writing to file trendingterms_table.csv.


In [32]:
client = LuminosoClient.connect('/')

client = client.change_path('/projects/d53m338v/prv28wdf')

subsets = client.get('subsets/stats')

subsets

subsets_to_remove = []
for subset in subsets:
    if subset['count'] < 150:
        subsets_to_remove.append(subset['subset'])

docs = []
while True:
    new_docs = client.get('docs', limit=25000, offset=len(docs))
    if new_docs:
        docs.extend(new_docs)
    else:
        break

for doc in docs:
    subset = []
    for doc_subset in doc['subsets']:
        if doc_subset not in subsets_to_remove:
            subset.append(doc_subset)
    doc['subsets'] = subset
        

client = LuminosoClient.connect()
proj_id = client.post(name='AT Kearney Glacier NP Reviews')['project_id']
client = client.change_path(proj_id)
batch = 0
batch_size = 10000
total_size = len(docs)
while batch < total_size:
    end = min(total_size, batch + batch_size)
    client.upload('docs', docs=docs[batch:end])
    batch += batch_size
client.post('docs/recalculate')